# Sensitivity Analysis for Unobserved Confounder with DML.

Here we experiment with sensitivity analysis under unobserved confounding, both manually and with [sensitivity.py](https://github.com/vsyrgkanis/dml_sensitivity_python).

## Partially Linear SEM

Consider the SEM
\begin{eqnarray*}
Y & := & \alpha D + \delta A + f_Y(X) + \epsilon_Y,  \\
D & := & \gamma A + f_D(X) + \epsilon_D, \\
A & : =  & f_A(X) + \epsilon_A, \\
X & := &  \epsilon_X,
\end{eqnarray*}
where, conditional on $X$, $\epsilon_Y, \epsilon_D, \epsilon_A$ are both mean zero
and mutually uncorrelated. We further normalize
$$
E[\epsilon_A^2] =1.
$$
The key structural
parameter is $\alpha$: $$\alpha = \partial_d Y(d)$$
where $$Y(d) := (Y: do (D=d)).$$

To give context to our example, we can interpret $Y$ as earnings,
$D$ as education, $A$ as ability, and $X$ as a set of observed background variables. In this example, we can interpret $\alpha$ as the returns to schooling.

We start by applying the partialling out operator to get rid of the $X$'s in all of the equations. Define the partialling out operation of any random vector $V$ with respect to another random vector $X$ as the residual that is left after subtracting the best predictor of $V$ given $X$:
$$\tilde V = V - E [V \mid X].$$  
If $f$'s are linear, we can replace $E [V \mid X]$
by linear projection.  After partialling out, we have a simplified system:
\begin{eqnarray*}
\tilde Y & := & \alpha \tilde D + \delta \tilde A + \epsilon_Y,  \\
\tilde D & := & \gamma \tilde A + \epsilon_D, \\
\tilde A & : = & \epsilon_A,
\end{eqnarray*}
where $\epsilon_Y$, $\epsilon_D$, and $\epsilon_A$ are uncorrelated.

Then the projection of $\tilde Y$ on $\tilde D$ recovers
$$
\beta = E [\tilde Y \tilde D]/ E [\tilde D^2] = \alpha +  \phi,
$$
where
$$
\phi =  \delta \gamma/ E \left[(\gamma^2 + \epsilon^2_D)\right],
$$
is the omitted confounder bias or omitted variable bias.

The formula follows from inserting the expression for $\tilde D$ into the definition of $\beta$ and then simplifying the resulting expression using the assumptions on the $\epsilon$'s.

We can use this formula to bound $\phi$ directly by making assumptions on the size of $\delta$
and $\gamma$.  An alternative approach can be based on the following characterization,
based on partial $R^2$'s.  This characterization essentially follows
from Cinelli and Hazlett, with the slight difference that we have adapted
the result to the partially linear model.

*Theorem* [Omitted Confounder Bias in Terms of Partial $R^2$'s]

In the partially linear SEM setting above,
$$
\phi^2 = \frac{R^2_{\tilde Y \sim \tilde A \mid \tilde D} R^2_{\tilde D \sim \tilde A} }{ (1 - R^2_{\tilde D \sim \tilde A}) } \
\frac{E \left[ (\tilde Y - \beta \tilde D)^2 \right] }{E \left[ ( \tilde D )^2 \right]},
$$
where $R^2_{V \sim W \mid X}$ denotes the population $R^2$ in the linear regression of $V$ on $W$, after partialling out linearly $X$ from $V$ and $W$.


Therefore, if we place bounds on how much of the variation in $\tilde Y$ and in $\tilde D$
the unobserved confounder $\tilde A$ is able to explain, we can bound the omitted confounder bias by $$\sqrt{\phi^2}.$$


# Empirical Example

We consider an empirical example based on data surrounding the Darfur war. Specifically, we are interested in the effect of having experienced direct war violence on attitudes towards peace. Data is described here
https://cran.r-project.org/web/packages/sensemakr/vignettes/sensemakr.html

The main outcome is attitude towards peace -- the peacefactor.
The key variable of interest is whether the responders were directly harmed (directlyharmed).
We want to know if being directly harmed in the conflict causes people to support peace-enforcing measures.
The measured confounders include female indicator, age, farmer, herder, voted in the past, and household size.
There is also a village indicator, which we will treat as fixed effect and partial it out before conducting
the analysis. The standard errors will be clustered at the village level.


## Outline

We will:
- mimic the partialling out procedure with machine learning tools;
- invoking sensitivity.py to compute $\phi^2$ and plot sensitivity results.


In [ ]:
# Import relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import patsy
import warnings
from sklearn.base import BaseEstimator
import statsmodels.formula.api as smf
warnings.simplefilter('ignore')
np.random.seed(1234)

In [ ]:
file = "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/darfur.csv"
data = pd.read_csv(file)
data.shape

## Preprocessing
Take out village fixed effects and run basic linear analysis

In [ ]:
# Get rid of village fixed effects
peacefactorR = smf.ols(formula="peacefactor ~ village", data=data).fit().resid
directlyharmedR = smf.ols(formula="directlyharmed ~ village", data=data).fit().resid
femaleR = smf.ols(formula="female ~ village", data=data).fit().resid
ageR = smf.ols(formula="age ~ village", data=data).fit().resid
farmerR = smf.ols(formula="farmer_dar ~ village", data=data).fit().resid
herderR = smf.ols(formula="herder_dar ~ village", data=data).fit().resid
pastvotedR = smf.ols(formula="pastvoted ~ village", data=data).fit().resid
hhsizeR = smf.ols(formula="hhsize_darfur ~ village", data=data).fit().resid

# Preliminary linear model analysis
model1 = smf.ols(formula="peacefactorR ~ directlyharmedR + femaleR + ageR + farmerR + herderR + pastvotedR + hhsizeR",
                 data=data).fit(cov_type='cluster', cov_kwds={'groups': data['village']})
print(model1.summary())

# Here we are clustering standard errors at the village level
model2 = smf.ols(formula="peacefactorR ~ femaleR + ageR + farmerR + herderR + pastvotedR + hhsizeR",
                 data=data).fit(cov_type='cluster', cov_kwds={'groups': data['village']})
print(model2.summary())

model3 = smf.ols(formula="directlyharmedR ~ femaleR + ageR + farmerR + herderR + pastvotedR + hhsizeR",
                 data=data).fit(cov_type='cluster', cov_kwds={'groups': data['village']})
print(model3.summary())

## Lasso for partialling out controls

Run the following commands to install hdmpy for rigorous lasso:

```
!pip install multiprocess
!git clone https://github.com/maxhuppertz/hdmpy.git
```

In [ ]:
!pip install multiprocess
!git clone https://github.com/maxhuppertz/hdmpy.git

In [ ]:
import hdmpy


class RLasso(BaseEstimator):

    def __init__(self, *, post=True):
        self.post = post

    def fit(self, X, y):
        self.rlasso_ = hdmpy.rlasso(X, y, post=self.post)
        return self

    def predict(self, X):
        return np.array(X) @ np.array(self.rlasso_.est['beta']).flatten() + np.array(self.rlasso_.est['intercept'])


def lasso_model():
    return RLasso(post=False)

In [ ]:
Z = np.column_stack((femaleR, ageR, farmerR, herderR, pastvotedR, hhsizeR))
Z = pd.DataFrame(Z, columns=['femaleR', 'ageR', 'farmerR', 'herderR', 'pastvotedR', 'hhsizeR'])
# Interactions of 3 degrees
controls = patsy.dmatrix('0 + (femaleR + ageR + farmerR + herderR + pastvotedR + hhsizeR)**3',
                         Z, return_type='dataframe')


In [ ]:
resY = peacefactorR - lasso_model().fit(controls, peacefactorR).predict(controls)
resD = directlyharmedR - lasso_model().fit(controls, directlyharmedR).predict(controls)
print(("Controls explain the following fraction of variance of Outcome", 1 - np.var(resY) / np.var(peacefactorR)))
print(("Controls explain the following fraction of variance of Treatment", 1 - np.var(resD) / np.var(directlyharmedR)))

dml_data = pd.DataFrame({'resY': resY, 'resD': resD, 'village': data['village']})
dml_model = smf.ols(formula="resY ~ resD", data=dml_data).fit(cov_type='cluster',
                                                              cov_kwds={'groups': dml_data['village']})
print(dml_model.summary())

## Manual Bias Analysis

In [ ]:
# Main estimate
beta = dml_model.params[1]

# Hypothetical values of partial R2s
R2_YC = 0.16
R2_DC = 0.01

# Elements of the bias equation
kappa = (R2_YC * R2_DC) / (1 - R2_DC)
variance_ratio = np.mean(dml_model.resid**2) / np.mean(resD**2)

# Compute square bias
BiasSq = kappa * variance_ratio

# Compute absolute value of the bias
print("absolute value of the bias:", np.sqrt(BiasSq))

# Plotting
gridR2_DC = np.arange(0, 0.301, 0.001)
gridR2_YC = kappa * (1 - gridR2_DC) / gridR2_DC
gridR2_YC = np.where(gridR2_YC > 1, 1, gridR2_YC)

plt.plot(gridR2_DC, gridR2_YC, color='red')
plt.xlabel('Partial R2 of Treatment with Confounder')
plt.ylabel('Partial R2 of Outcome with Confounder')
plt.title(f'Combo of R2 such that |Bias| < {np.round(np.sqrt(BiasSq), decimals=4)}')
plt.show()

## Bias Analysis with some automated functions.

We now automate the DML process and pass the estimates to functions to automate the sensitivity analysis. This is done in the R package sensmakr, which does not exist in Python.

In [ ]:
def dml(X, D, y, modely, modeld, *, nfolds, classifier=False, cluster=True, clu=None):
    '''
    DML for the Partially Linear Model setting with cross-fitting

    Input
    -----
    X: the controls
    D: the treatment
    y: the outcome
    modely: the ML model for predicting the outcome y
    modeld: the ML model for predicting the treatment D
    nfolds: the number of folds in cross-fitting
    classifier: bool, whether the modeld is a classifier or a regressor

    clu: df column to cluster by
    cluster: bool, whether to use clustered standard errors

    Output
    ------
    point: the point estimate of the treatment effect of D on y
    stderr: the standard error of the treatment effect
    yhat: the cross-fitted predictions for the outcome y
    Dhat: the cross-fitted predictions for the treatment D
    resy: the outcome residuals
    resD: the treatment residuals
    epsilon: the final residual-on-residual OLS regression residual
    '''

    if nfolds > 1:
        cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)  # shuffled k-folds
        yhat = cross_val_predict(modely, X, y, cv=cv, n_jobs=-1)  # out-of-fold predictions for y
        # out-of-fold predictions for D
        # use predict or predict_proba dependent on classifier or regressor for D
        if classifier:
            Dhat = cross_val_predict(modeld, X, D, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
        else:
            Dhat = cross_val_predict(modeld, X, D, cv=cv, n_jobs=-1)
    elif nfolds == -1:
        yhat = modely.fit(X, y).predict(X)
        if classifier:
            Dhat = modeld.fit(X, D).predict_proba(X)
        else:
            Dhat = modeld.fit(X, D).predict(X)

    # calculate outcome and treatment residuals
    resy = y - yhat
    resD = D - Dhat

    if cluster:
        # final stage ols clustered
        dml_data = pd.DataFrame({'resY': resY, 'resD': resD, 'cluster': clu})
    else:
        # final stage ols nonclustered
        dml_data = pd.DataFrame({'resY': resY, 'resD': resD})

    if cluster:
        # clustered standard errors
        ols_mod = smf.ols(formula='resY ~ 1 + resD', data=dml_data)
        ols_mod = ols_mod.fit(cov_type='cluster', cov_kwds={"groups": dml_data['cluster']})
    else:
        # regular ols
        ols_mod = smf.ols(formula='resY ~ 1 + resD', data=dml_data).fit()

    point = ols_mod.params[1]
    stderr = ols_mod.bse[1]
    epsilon = ols_mod.resid

    return point, stderr, yhat, Dhat, resy, resD, epsilon

In [ ]:
def summary(point, stderr, yhat, Dhat, resy, resD, epsilon, X, D, y, *, name):
    '''
    Convenience summary function that takes the results of the DML function
    and summarizes several estimation quantities and performance metrics.
    '''
    return pd.DataFrame({'estimate': point,  # point estimate
                         'stderr': stderr,  # standard error
                         'lower': point - 1.96 * stderr,  # lower end of 95% confidence interval
                         'upper': point + 1.96 * stderr,  # upper end of 95% confidence interval
                         'rmse y': np.sqrt(np.mean(resy**2)),  # RMSE of model that predicts outcome y
                         'rmse D': np.sqrt(np.mean(resD**2))  # RMSE of model that predicts treatment D
                         }, index=[name])


In [ ]:
def dml_sensitivity_bounds_single(res, eta_ysq, eta_asq, alpha=None, inds=None):
    ''' Sensitivity analysis, specialized for the partially linear DML moment
    E[(yres - theta * Tres) * Tres]. `est` is a `LinearDML` estimator fitted
    with `cache_values=True` so that residuals are being stored after fitting.
    '''
    if inds is None:
        inds = np.arange(res[0].shape[0])
    yres, Tres = res
    yres, Tres = yres[inds], Tres[inds]
    nusq = np.mean(Tres ** 2)
    theta = np.mean(yres * Tres) / nusq
    sigmasq = np.mean((yres - Tres * theta)**2)
    S = np.sqrt(sigmasq / nusq)
    Casq = eta_asq / (1 - eta_asq)
    Cgsq = eta_ysq
    error = S * np.sqrt(Casq * Cgsq)

    if alpha is None:
        return theta - error, theta + error

    psi_theta = (yres - Tres * theta) * Tres / nusq
    psi_sigmasq = (yres - Tres * theta)**2 - sigmasq
    psi_nusq = Tres**2 - nusq

    phi_plus = psi_theta
    phi_plus += (np.sqrt(Casq * Cgsq) / (2 * S)) * (-(sigmasq / (nusq**2)) * psi_nusq + (1 / nusq) * psi_sigmasq)
    stderr_plus = np.sqrt(np.mean(phi_plus**2) / phi_plus.shape[0])

    phi_minus = psi_theta
    phi_minus -= (np.sqrt(Casq * Cgsq) / (2 * S)) * (-(sigmasq / (nusq**2)) * psi_nusq + (1 / nusq) * psi_sigmasq)
    stderr_minus = np.sqrt(np.mean(phi_minus**2) / phi_minus.shape[0])
    return theta - error, stderr_minus, theta + error, stderr_plus


def dml_sensitivity_contours_single(res, a_upper, y_upper, alpha=None, inds=None):
    ''' Specialized for linear DML. Sensitivity bounds contour plots based on a many trained doubly robust
    average moment models. If `alpha` is float, incorporates sampling uncertainty
    at the `alpha` level. Sensitivity parameter `eta_ysq` ranges in `[0, y_upper]`
    and parameter `eta_asq` ranges in `[0, a_upper]`.
    '''
    xlist = np.linspace(0, a_upper, 100)
    ylist = np.linspace(0, y_upper, 100)
    X, Y = np.meshgrid(xlist, ylist)
    Zlower = np.zeros(X.shape)
    Zupper = np.zeros(X.shape)
    for itx in np.arange(X.shape[1]):
        for ity in np.arange(X.shape[0]):
            if alpha is None:
                l, u, = dml_sensitivity_bounds_single(res, Y[ity, itx], X[ity, itx], alpha=alpha, inds=inds)
            else:
                l, _, u, _ = dml_sensitivity_bounds_single(res, Y[ity, itx], X[ity, itx], alpha=alpha, inds=inds)
            Zlower[ity, itx] = l
            Zupper[ity, itx] = u

    return X, Y, Zlower, Zupper

In [ ]:
# DML with RLasso
modely = make_pipeline(StandardScaler(), lasso_model())
modeld = make_pipeline(StandardScaler(), lasso_model())

# Run DML model with no crossfitting (change nfolds to >1 to use crossfitting)
result_lasso = dml(controls, directlyharmedR, peacefactorR, modely, modeld,
                   nfolds=-1, classifier=False, clu=data['village'], cluster=True)
table_lasso = summary(*result_lasso, controls, directlyharmedR, peacefactorR, name='RLasso')
print(table_lasso)

resY_lasso, resD_lasso = result_lasso[4], result_lasso[5]
print(("Controls explain the following fraction of variance of Outcome",
       max(1 - np.var(resY_lasso) / np.var(peacefactorR), 0)))
print(("Controls explain the following fraction of variance of Treatment",
       max(1 - np.var(resD_lasso) / np.var(directlyharmedR), 0)))


In [ ]:
res = [resY_lasso, resD_lasso]

In [ ]:
print("[beta - phi, beta + phi]: ", dml_sensitivity_bounds_single(res, 0.16, 0.01))

In [ ]:
eta_asq, eta_ysq, lower, upper = dml_sensitivity_contours_single(res, 0.4, 0.4, alpha=0.05)
# Sensitivity parameter `eta_ysq` = R2_{Y ~ A|D} ranges in `[0, y_upper]`
# and `eta_asq` = R2_{D ~ A} ranges in `[0, a_upper]`.

In [ ]:
contours = plt.contour(eta_asq, eta_ysq, lower, 6, linestyles='-', colors='blue')
plt.clabel(contours, inline=True, fontsize=8)
plt.title('Lower limit')
plt.xlabel('Partial R2 of confounder(s) with the treatment')
plt.ylabel('Partial R2 of confounder(s) with the outcome')

## Random Forest for partialling out

The following code does DML with clustered standard errors by village

In [ ]:
# DML with Random Forests. RFs don't require scaling but we do it for consistency
modely = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100, min_samples_leaf=5, random_state=123))
modeld = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100, min_samples_leaf=5, random_state=123))

# Run DML model with nfolds folds of cross-fitting (computationally intensive)
result_RF = dml(Z, directlyharmedR, peacefactorR, modely, modeld, nfolds=10,
                classifier=False, clu=data['village'], cluster=True)
table_RF = summary(*result_RF, Z, directlyharmedR, peacefactorR, name='RF')
print(table_RF)

resY_RF, resD_RF = result_RF[4], result_RF[5]
print(("Controls explain the following fraction of variance of Outcome",
       max(1 - np.var(resY_RF) / np.var(peacefactorR), 0)))
print(("Controls explain the following fraction of variance of Treatment",
       max(1 - np.var(resD_RF) / np.var(directlyharmedR), 0)))

## Bias Analysis

If we want reduce the uncertainty from sample splitting, we can re-run our $nfolds$ cross-fitting and aggregate with the median estimates. With 10 folds as above, there shouldn't be too much variance anyway, but we employ the following aggregation procedure anyway.


In [ ]:
import scipy


def dml_sensitivity_bounds(est_list, eta_ysq, eta_asq, alpha=None, inds=None):
    if alpha is None:
        lower, upper = zip(*[dml_sensitivity_bounds_single(est, eta_ysq, eta_asq, alpha=alpha, inds=inds)
                             for est in est_list])
        return np.median(lower), np.median(upper)
    else:
        lower, std_lower, upper, std_upper = zip(*[dml_sensitivity_bounds_single(est, eta_ysq, eta_asq,
                                                                                 alpha=alpha, inds=inds)
                                                   for est in est_list])
        std_lower = np.array(std_lower)
        std_upper = np.array(std_upper)
        lower = np.median(lower) - scipy.stats.norm.ppf(1 - alpha) * np.sqrt(np.median(std_lower**2) + np.var(lower))
        upper = np.median(upper) + scipy.stats.norm.ppf(1 - alpha) * np.sqrt(np.median(std_upper**2) + np.var(upper))
        return lower, upper


def dml_sensitivity_contours(est_list, a_upper, y_upper, alpha=None, inds=None):
    ''' Specialized for linear DML. Sensitivity bounds contour plots based on a many trained doubly robust
    average moment models. If `alpha` is float, incorporates sampling uncertainty
    at the `alpha` level. Sensitivity parameter `eta_ysq` ranges in `[0, y_upper]`
    and parameter `eta_asq` ranges in `[0, a_upper]`.
    '''
    xlist = np.linspace(0, a_upper, 100)
    ylist = np.linspace(0, y_upper, 100)
    X, Y = np.meshgrid(xlist, ylist)
    Zlower = np.zeros(X.shape)
    Zupper = np.zeros(X.shape)
    for itx in np.arange(X.shape[1]):
        for ity in np.arange(X.shape[0]):
            l, u = dml_sensitivity_bounds(est_list, Y[ity, itx], X[ity, itx], alpha=alpha, inds=inds)
            Zlower[ity, itx] = l
            Zupper[ity, itx] = u

    return X, Y, Zlower, Zupper

In [ ]:
# redefine models without setting state -- random_state controls train_test_split downstream
modely = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100, min_samples_leaf=5))
modeld = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100, min_samples_leaf=5))
res_RF_list = []
for i in range(5):
    result_RF = dml(Z, directlyharmedR, peacefactorR, modely, modeld, nfolds=10,
                    classifier=False, clu=data['village'], cluster=True)
    table_RF = summary(*result_RF, Z, directlyharmedR, peacefactorR, name='RF')
    resY_RF, resD_RF = result_RF[4], result_RF[5]
    res_RF = [resY_RF, resD_RF]
    res_RF_list.append(res_RF)

In [ ]:
print("[beta - phi, beta + phi]: ", dml_sensitivity_bounds(res_RF_list, 0.16, 0.01))
alp = 0.05
print(f"With Sampling Uncertainty at the {alp} level:", dml_sensitivity_bounds(res_RF_list, .16, .01, alpha=.05))

In [ ]:
alp = 0.05
eta_asq, eta_ysq, lower, upper = dml_sensitivity_contours(res_RF_list, 0.4, 0.4, alpha=alp)
contours = plt.contour(eta_asq, eta_ysq, lower, 6, linestyles='-', colors='blue')
plt.clabel(contours, inline=True, fontsize=8)
plt.title('Lower limit')
plt.xlabel('Partial R2 of confounder(s) with the treatment')
plt.ylabel('Partial R2 of confounder(s) with the outcome')